### Carga de librerías necesarias

Se importan los principales paquetes y módulos necesarios para el ajuste fino del modelo de lenguaje:

- `transformers`: Para la carga del modelo base, tokenizador y configuración del entrenamiento.
- `peft`: Librería para técnicas de ajuste fino eficiente como LoRA.
- `torch` y `os`: Manejo de operaciones básicas con PyTorch y sistema de archivos.
- `datasets`: Para manipulación y carga del conjunto de datos.
- `trl`: Se importa exclusivamente el módulo `SFTTrainer` (Supervised Fine-Tuning Trainer).
- `pandas` y `pyarrow`: Utilizadas para tratamiento y compatibilidad de datos tabulares.
- `re`: Expresiones regulares utilizadas para procesamiento textual.
- `accelerate`: Herramientas para facilitar la distribución del modelo entre dispositivos.
- `json`: Para leer y transformar estructuras de datos en formato JSON.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch
from datasets import load_datase, Dataset
from trl import SFTTrainer 
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
import re
from accelerate import init_empty_weights, infer_auto_device_map
import json

### Preparación y carga del conjunto de datos

Se define la ruta del modelo base (`Mistral-7B-Instruct-v0.2`) y el nombre del nuevo modelo especializado (`Oswestry-Instruct`).

A continuación, se implementa una función `load_dataset()` que:
- Carga el archivo JSON con los datos generados para el ajuste fino.
- Formatea cada entrada conforme al formato esperado por el modelo que ocupamos (formato de conversación con tokens especiales `<s>[INST] ... [/INST] ... </s>`).
- Devuelve un `Dataset` de HuggingFace estructurado para el entrenamiento.

El archivo cargado contiene las entrevistas médico-paciente generadas con ChatGPT junto con la correspondiente evaluación esperada también generada sintéticamente. 

Posteriormente, el conjunto de datos completo se divide automáticamente en dos subconjuntos:
- **80 % para entrenamiento** (`train_dataset`)
- **20 % para validación** (`eval_dataset`)

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "Oswestry-Instruct"

def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    formatted_data = []
    for item in data:
        text = f"<s>[INST] {item['instruction']} {item['input']} [/INST] {item['output']} </s>"
        formatted_data.append({"text": text})

    return Dataset.from_list(formatted_data)

dataset_path = "Aquí va la ruta al archivo JSON con los datos de ajuste fino"  # Reemplazar con la ruta correcta

full_dataset = load_dataset(dataset_path)
dataset = full_dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset["train"]
eval_dataset = dataset["test"]

### Autenticación con Hugging Face

Para poder cargar y subir modelos desde y hacia la plataforma Hugging Face, es necesario autenticarse mediante un token personal de acceso.

Dentro de Kaggle, se accede de forma segura al token usando la clase `UserSecretsClient`, que permite recuperar secretos almacenados de forma privada.

- Se obtiene el token de acceso personal mediante `user_secrets.get_secret("HUGGINGFACE_TOKEN")`.
- Se realiza el login automático usando el comando `huggingface-cli login`.

Esta autenticación es un paso previo obligatorio para utilizar repositorios privados o realizar `push_to_hub()` al finalizar el entrenamiento del modelo.

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
!huggingface-cli login --token $secret_hf

### Carga del modelo base y configuración de cuantización

En esta sección se carga el modelo base `Mistral-7B-Instruct-v0.2`, una variante preentrenada orientada a tareas instruccionales. Para hacer viable el ajuste fino en entornos con recursos limitados como Kaggle, se aplica cuantización extrema a 4 bits usando la librería `BitsAndBytes`.

- **Cuantización a 4 bits (`load_in_4bit=True`)**: reduce drásticamente el uso de memoria, permitiendo entrenar modelos grandes.
- **Tipo de cuantización `nf4`**: formato que mantiene buena precisión incluso en baja resolución.
- **Cómputo en `bfloat16`**: mejora el aprovechamiento de la GPU sin degradar significativamente el rendimiento.
- **`device_map="auto"`**: distribuye automáticamente las capas del modelo entre los dispositivos disponibles.

Posteriormente, se realiza la configuración del modelo para entrenamiento eficiente:
- Se desactiva la caché con `use_cache = False` para permitir el uso de checkpointing de gradientes.
- Se activa `gradient_checkpointing`, una técnica que reduce el uso de memoria durante el entrenamiento.
- Se carga el tokenizador correspondiente al modelo, ajustando el padding y añadiendo el token de fin de secuencia (`eos_token`) como marcador de relleno.


In [ ]:

# Configuración de la cuantización
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= True,
)

# Cargamos el modelo base con la configuración de cuantización
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Carga del tokenizador
tokenizer = AutoTokenizer.from_pretrained(
    base_model, 
    trust_remote_code=True,
    use_fast=False 
)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

### Preparación del modelo para ajuste fino con LoRA

1. `prepare_model_for_kbit_training(model)`: prepara el modelo previamente cuantizado para ser entrenado con técnicas PEFT, asegurando compatibilidad con estructuras de baja precisión.

2. **Configuración LoRA (`LoraConfig`)**:
   - `r=64`: rango de la descomposición de matrices, controlando la capacidad de adaptación del modelo.
   - `lora_alpha=16`: factor de escalado de las actualizaciones LoRA.
   - `lora_dropout=0.1`: probabilidad de aplicar dropout a las capas LoRA.
   - `bias="none"`: no se actualizan los sesgos durante el entrenamiento.
   - `task_type="CAUSAL_LM"`: especifica que se está trabajando con un modelo de lenguaje causal.
   - `target_modules`: define las proyecciones de atención sobre las cuales se aplicará la adaptación (claves, valores, consultas, salida y compuerta).

3. `get_peft_model(...)`: aplica la configuración LoRA al modelo cargado, encapsulándolo en una nueva estructura entrenable con un número reducido de parámetros.


In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

### Definición de los hiperparámetros de entrenamiento

Se establece la configuración de entrenamiento mediante la clase `TrainingArguments`, adaptada a las limitaciones computacionales del entorno. 

- **Epochs**: Solo se realiza una época de entrenamiento (`num_train_epochs=1`), ya que se parte de un modelo previamente preentrenado y se busca una adaptación eficiente con un dataset de tamaño reducido.
- **Batch Size**: Se utiliza un tamaño de lote de 4 (`per_device_train_batch_size=4`) y se activa la acumulación de gradientes (`gradient_accumulation_steps=1`) para mejorar el uso de memoria.
- **Optimizador**: Se selecciona `paged_adamw_32bit`, diseñado para eficiencia en modelos cuantizados.
- **Precisión Mixta**: Se activa tanto `fp16=True` como `bf16=True` para maximizar la compatibilidad y el rendimiento en entornos con soporte mixto.
- **Evaluación**: Se realiza una evaluación periódica cada 50 pasos (`eval_steps=50`) y se registran métricas clave en la plataforma Weights & Biases (`report_to="wandb"`).
- **Programador de tasa de aprendizaje**: Se opta por un scheduler constante con una fase inicial de calentamiento (`warmup_ratio=0.03`) para estabilizar el entrenamiento.
- **Ajustes Triviales**: Se incluye el recorte de gradiente (`max_grad_norm=0.3`), agrupación por longitud de secuencia (`group_by_length=True`) y un directorio específico para logs (`logging_dir="./logs"`).

In [ ]:
 #Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    evaluation_strategy="steps",   
    eval_steps=50,                
    logging_dir="./logs",          
    report_to="wandb",             
)

### Configuración del entrenador supervisado (SFTTrainer)

Se inicializa el entrenador supervisado mediante la clase `SFTTrainer` de la librería `trl`, diseñada específicamente para realizar ajustes finos en modelos de lenguaje utilizando el enfoque PEFT (Parameter-Efficient Fine-Tuning).

Parámetros principales:
- **`model`**: Modelo base ya preparado con la configuración LoRA aplicada.
- **`train_dataset` y `eval_dataset`**: Conjuntos de datos de entrenamiento y validación, previamente procesados y divididos.
- **`peft_config`**: Configuración LoRA que define el rango, el dropout y los módulos del modelo a ajustar.
- **`tokenizer`**: Tokenizador correspondiente al modelo utilizado.
- **`args`**: Conjunto de hiperparámetros definidos en `TrainingArguments`.
- **`dataset_text_field`**: Se especifica que el campo "text" contiene las entradas ya formateadas del dataset.
- **`max_seq_length`**: No se impone una longitud máxima explícita, permitiendo usar el valor por defecto del modelo.
- **`packing`**: Se desactiva la agrupación de múltiples muestras en una sola secuencia (desactivado por defecto para tareas de generación).

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,  
    eval_dataset=eval_dataset, 
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

### Entrenamiento, guardado y despliegue del modelo

- **Autenticación en Weights & Biases (W&B)**: Se realiza el login en la plataforma de monitorización con una clave API segura obtenida desde `kaggle_secrets`. Esto permite registrar métricas clave durante el entrenamiento.

- **Entrenamiento del modelo**: Se lanza el ajuste fino mediante `trainer.train()`, utilizando los parámetros, datasets y modelo previamente definidos.

- **Guardado del modelo ajustado**: Una vez completado el entrenamiento, se guarda el modelo con `save_pretrained(new_model)`, conservando únicamente los pesos LoRA especializados.

- **Configuración para inferencia**: Se vuelve a activar la caché de resultados (`use_cache = True`) y se establece el modelo en modo evaluación (`model.eval()`).

- **Publicación en Hugging Face Hub**:
  - El modelo especializado se sube a Hugging Face Hub con `push_to_hub()`.
  - También se sube el tokenizer correspondiente para garantizar compatibilidad en el uso futuro del modelo.

Con esta última celda se concluye el proceso de ajuste fino.

In [ ]:
import wandb
secret_value_1 = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=secret_value_1)
trainer.train()
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()
trainer.model.push_to_hub(new_model)
tokenizer.push_to_hub("nombre_modelo")